In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/Dominik/R-workspace/cecelia/inst')

# MacOS
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/USER_data/Dominik/CECELIA_BACKUP/CV5iNI/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/IksEJv/')
#im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
#im_path = os.path.join(zero_dir, 'ccidAfCorrected.zarr')
im_path = os.path.join(zero_dir, 'ccidSlidingWindow.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'IksEJv')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'


In [4]:
channel_names = [
    # 'EB', 'T-A', 'T-B', 'AF', 'gen'
    'One'
]

In [5]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    downsample_z = False, squeeze = False
)

<zarr.hierarchy.Group '/' read-only>
[81, 4, 11, 632, 538]
[538, 632, 11, 4, 81]
['X', 'Y', 'Z', 'C', 'T']
[81, 4, 11, 632, 538]


In [6]:
import py.zarr_utils as zarr_utils

im, _ = zarr_utils.open_as_zarr(im_path, as_dask = True)

<zarr.hierarchy.Group '/' read-only>


In [7]:
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils
import py.correction_utils as correction_utils
import py.slice_utils as slice_utils

In [8]:
# get OME-XML
omexml = ome_xml_utils.parse_meta(im_path)

# create dim utils for image
dim_utils = DimUtils(omexml, use_channel_axis = True)
dim_utils.calc_image_dimensions(im[0].shape)

[81, 4, 11, 632, 538]
[538, 632, 11, 4, 81]
['X', 'Y', 'Z', 'C', 'T']


In [9]:
import py.correction_utils as correction_utils
import py.slice_utils as slice_utils

In [10]:
slices = [slice(None) for _ in range(len(im[0].shape))]
#slices[dim_utils.dim_idx('C')] = slice(2, 3, 1)
#slices[dim_utils.dim_idx('Z')] = slice(4, 5, 1)
#slices[dim_utils.dim_idx('T')] = slice(49, 50, 1)
slices = tuple(slices)

In [34]:
af_combinations = {
    2: {
        'divisionChannels': [3],
        'channelPercentile': 80,
        'correctionPercentile': 40,
        'correctionMode': 'divide',
        'medianFilter': 0,
        'generateInverse': True,
        'topHatRadius': 40
    }}

a = correction_utils.af_correct_image(
    im[0], af_combinations, dim_utils = dim_utils
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:39<00:00,  2.04it/s]


In [36]:
napari_utils.viewer.add_image(a, channel_axis=dim_utils.dim_idx('C'), blending='additive', scale=napari_utils.im_scale)

[<Image layer 'Image' at 0x16872b430>,
 <Image layer 'Image [1]' at 0x16d1f42e0>,
 <Image layer 'Image [2]' at 0x168a1e520>,
 <Image layer 'Image [3]' at 0x16cf6cb50>,
 <Image layer 'Image [4]' at 0x16cf6cbb0>]

In [ ]:
channel_slices = [slice(None) for _ in range(len(im[0].shape))]
channel_slices[dim_utils.dim_idx('C')] = slice(2, 3, 1)
channel_slices = tuple(channel_slices)

In [ ]:
time_slices = [slice(None) for _ in range(len(im[0].shape))]
time_slices[dim_utils.dim_idx('T')] = slice(49, 50, 1)
time_slices = tuple(time_slices)

In [ ]:
import pyclesperanto_prototype as cle

In [ ]:
# can you extract the rod like filaments or the round cells?
from matplotlib import pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

im_out = correction_utils.apply_top_hat(im[0][channel_slices], dim_utils, radius=10)

ax[0].imshow(np.max(np.squeeze(im[0][slices]), axis=0), cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

ax[1].imshow(np.max(np.squeeze(im_out[time_slices]), axis=0), cmap='gray')
ax[1].set_title('Filtered')
ax[1].axis('off')

fig.tight_layout()

In [ ]:
import py.correction_utils as correction_utils
import py.slice_utils as slice_utils

In [ ]:
slices = [slice(None) for _ in range(len(im[0].shape))]
slices[dim_utils.dim_idx('C')] = slice(2, 3, 1)
slices = tuple(slices)

In [ ]:
corrected = im[0][slices].copy()
corrected = correction_utils.apply_rolling_ball(corrected, dim_utils, radius=40)

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2)

ax[0].imshow(im[0][slices][5, 0, 0, :, :], cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

ax[1].imshow(corrected[5, 0, 0, :, :], cmap='gray')
ax[1].set_title('Background')
ax[1].axis('off')

fig.tight_layout()

In [ ]:
import napari

viewer = napari.viewer.Viewer()

In [ ]:
viewer.add_image(im[0][slices], channel_axis=1, blending='additive', colormap='green')
viewer.add_image(corrected, channel_axis=1, blending='additive', colormap='magenta')
viewer.add_image(im[0][slices] - corrected, channel_axis=1, blending='additive', colormap='magenta')

In [ ]:
for i in range(0, 5, 1):
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    
    im = axes[0].imshow(images[i])
    fig.colorbar(im, ax=axes[0])
    axes[0].set_title("Original")
    
    im = axes[1].imshow(images_transformed[i])
    fig.colorbar(im, ax=axes[1])
    axes[1].set_title("Corrected")
    fig.suptitle(f"frame {i}")
    
    fig.tight_layout()

In [ ]:
import py.correction_utils as correction_utils

In [ ]:
import importlib
importlib.reload(correction_utils)

In [ ]:
a = correction_utils.af_correct_image(
    im[0], {
        0: {'divisionChannels': [], 'rollingBallRadius': 0},
        1: {'divisionChannels': [], 'rollingBallRadius': 40, 'rollingBallPadding' : 4},
    }, dim_utils = dim_utils
)

In [ ]:
import napari

viewer = napari.Viewer()

In [ ]:
viewer.add_image(a, channel_axis = dim_utils.dim_idx('C'))

In [ ]:
# crop zero edges
# https://stackoverflow.com/a/39466129
def crop_zero_edges(im):
    # argwhere will give you the coordinates of every non-zero point
    true_points = np.argwhere(im)
    
    # take the smallest points and use them as the top left of your crop
    top_left = true_points.min(axis=0)
    
    # take the largest points and use them as the bottom right of your crop
    bottom_right = true_points.max(axis=0)

    return {'tl': top_left, 'br': bottom_right}

In [ ]:
rolling_ball_radius = 40
rolling_ball_padding = 4

In [ ]:
import py.slice_utils as slice_utils

In [ ]:
# get slices
slices = slice_utils.create_slices(im.shape, dim_utils)
for i, x in enumerate(slices):
    x = list(x)
    x[dim_utils.dim_idx('C')] = slice(1, 2, 1)
    slices[i] = tuple(x)

In [ ]:
slices[1:2]

In [ ]:
import skimage
from skimage.restoration import rolling_ball
from skimage.morphology import disk

In [ ]:
cur_slices

In [ ]:
crop_slices

In [ ]:
im[cur_slices][crop_slices].shape

In [ ]:
# corrected_data = corrected_data - rolling_ball(corrected_data, radius = rolling_ball_radius)
# TODO go through in 2D as 3D seems a bit much
footprint = skimage.morphology.disk(rolling_ball_radius)

def apply_2D_rolling_ball(im, slices, dim_utils, radius = 40, padding = 4):
    im_to_process = np.squeeze(im[cur_slices])

    edges = crop_zero_edges(im_to_process)
    crop_slices = (
        slice(edges['tl'][0] + padding,
              edges['br'][0] - padding, 1),
        slice(edges['tl'][1] + padding,
              edges['br'][1] - padding, 1)
    )

    slices = list(slices)
    slices[dim_utils.dim_idx('Y')] = crop_slices[0]
    slices[dim_utils.dim_idx('X')] = crop_slices[1]
    slices = tuple(slices)
    
    im[slices] = rolling_ball(
        im_to_process[crop_slices], radius = radius)

# go through slices
for cur_slices in slices[1:2]:
    if dim_utils.is_3D():
        # go through Z
        for z in range(dim_utils.dim_val('Z')):
            print(f'> {z}')
            
            cur_slices = list(cur_slices)
            cur_slices[dim_utils.dim_idx('Z')] = slice(z, z+1, 1)
            cur_slices = tuple(cur_slices)
            
            apply_2D_rolling_ball(
                im, cur_slices, dim_utils, rolling_ball_radius, rolling_ball_padding)
    else:
        apply_2D_rolling_ball(
            im, cur_slices, dim_utils, rolling_ball_radius, rolling_ball_padding)

In [ ]:

import math

radius = 40
padding = 4



im_to_process = np.squeeze(im[slices])
edges = crop_zero_edges(im_to_process)
crop_slices = (
    slice(edges['tl'][0] + padding, edges['br'][0] - padding, 1),
    slice(edges['tl'][1] + padding, edges['br'][1] - padding, 1)
)

rolling_im = rolling_ball(im_to_process[crop_slices], radius = radius)

In [ ]:
dim_utils.is_timeseries()

In [ ]:
# corrected_data = corrected_data - rolling_ball(corrected_data, radius = rolling_ball_radius)
# TODO go through in 2D as 3D seems a bit much
footprint = skimage.morphology.disk(rolling_ball_radius)

if dim_utils.is_timeseries():

im_to_process = np.squeeze(im[slices])
edges = crop_zero_edges(im_to_process)
crop_slices = (
    slice(edges['tl'][0] + padding, edges['br'][0] - padding, 1),
    slice(edges['tl'][1] + padding, edges['br'][1] - padding, 1)
)

rolling_im = rolling_ball(im_to_process[crop_slices], radius = radius)

In [ ]:
# display results
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4),
                         sharex=True, sharey=True)

ax = axes.ravel()

# compute corrected image
corrected_im = np.zeros_like(im_to_process)
corrected_im[crop_slices] = im_to_process[crop_slices] - rolling_im

ax[0].imshow(im_to_process, cmap=plt.cm.gray)
ax[0].axis('Off')
ax[0].set_title('Image', fontsize=20)

ax[1].imshow(rolling_im, cmap=plt.cm.gray)
ax[1].axis('Off')
ax[1].set_title('Background', fontsize=20)

ax[2].imshow(corrected_im, cmap=plt.cm.gray)
ax[2].axis('Off')
ax[2].set_title('Corrected', fontsize=20)

fig.tight_layout()
plt.show()